In [ ]:
from sim_util import preview_from_definition
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import colors

%matplotlib inline
plt.rcParams['figure.figsize'] = [8,8]

In [ ]:
# simulate one plane of images

z_plane = 255
preview = preview_from_definition('/scratch/hoerl/sim_tissue3/sim3_2.json', z_plane)
print('Done.')

In [ ]:
# plot the planes
# for different lambda, illum, x_pos

for k in sorted(set(map(lambda x: x[::2], preview.keys()))):
    lam, xpos = k
    print(k)
    k_right = (lam, True, xpos)
    k_left = (lam, False, xpos)
    v_r = preview[k_right]
    v_l = preview[k_left]
    
    plt.figure()
    sig_r, desc_r = v_r
    sig_l, desc_l = v_l
    
    sig = np.stack((sig_r.squeeze(), sig_l.squeeze(), np.zeros_like(sig_r.squeeze())), 2)    
    
    # crude intensity scaling
    sig *= 5
    plt.imshow(sig)
    plt.show()